In [1]:
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import datetime
from sklearn.metrics import classification_report

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
LENGTH = 2000
ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [3]:
log_dir = "/tmp/tensorboard/{}".format(ts)
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)

In [4]:
pikachu = pd.read_csv('data/pikachu_df.csv').sample(n=LENGTH, random_state=1)
bulbasaur = pd.read_csv('data/bulbasaur_df.csv').sample(n=LENGTH, random_state=1)

In [5]:
pikachu

,RedRatio,GreenRatio,BlueRatio,Class
807,0.405,0.381,0.214,0
885,0.395,0.369,0.236,0
1685,0.366,0.366,0.267,0
983,0.381,0.381,0.238,0
2411,0.373,0.382,0.245,0
...,...,...,...,...
1744,0.395,0.363,0.242,0
449,0.338,0.392,0.270,0
550,0.389,0.362,0.249,0
2371,0.405,0.357,0.238,0


In [6]:
X = pd.concat([pikachu, bulbasaur])
y = X.pop('Class')

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [8]:
model = tf.keras.Sequential(
    tf.keras.layers.Dense(units=1, 
                          input_shape=[3], 
                          activation='sigmoid')
)

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics="accuracy"
)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(X_train, y_train,
    epochs=10,
    validation_split=0.1,
    callbacks=[tensorboard_callback],
)

Epoch 1/10
79/79 [==============================] - 0s 3ms/step - loss: 0.6448 - accuracy: 0.8643 - val_loss: 0.6273 - val_accuracy: 0.9571
Epoch 2/10
79/79 [==============================] - 0s 1ms/step - loss: 0.6115 - accuracy: 0.9687 - val_loss: 0.5968 - val_accuracy: 0.9536
Epoch 3/10
79/79 [==============================] - 0s 1ms/step - loss: 0.5815 - accuracy: 0.9742 - val_loss: 0.5682 - val_accuracy: 0.9571
Epoch 4/10
79/79 [==============================] - 0s 1ms/step - loss: 0.5531 - accuracy: 0.9742 - val_loss: 0.5410 - val_accuracy: 0.9643
Epoch 5/10
79/79 [==============================] - 0s 1ms/step - loss: 0.5268 - accuracy: 0.9758 - val_loss: 0.5164 - val_accuracy: 0.9607
Epoch 6/10
79/79 [==============================] - 0s 2ms/step - loss: 0.5019 - accuracy: 0.9722 - val_loss: 0.4924 - val_accuracy: 0.9643
Epoch 7/10
79/79 [==============================] - 0s 1ms/step - loss: 0.4789 - accuracy: 0.9758 - val_loss: 0.4721 - val_accuracy: 0.9571
Epoch 8/10
79/79 [==

In [11]:
y_pred = model.predict(X_test)

In [69]:
preds = [1 if x > 0.50 else 0 for x in y_pred]

In [71]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       601
           1       0.97      0.97      0.97       599

    accuracy                           0.97      1200
   macro avg       0.97      0.97      0.97      1200
weighted avg       0.97      0.97      0.97      1200



In [73]:
# Saved as SavedModel
tf.saved_model.save(model, "model/savedmodel_{}".format(ts))

INFO:tensorflow:Assets written to: model/savedmodel_20210108-190240/assets


# Convert model

In [77]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
model_path = "model/pika_bulba{}.tflite".format(ts)
open(model_path, "wb").write(tflite_model)
  
print("Model is {} bytes".format(os.path.getsize(model_path)))

INFO:tensorflow:Assets written to: /var/folders/b6/86znnmrd07v98wv0s_rgr4gh0000gn/T/tmpjmomkmfr/assets


INFO:tensorflow:Assets written to: /var/folders/b6/86znnmrd07v98wv0s_rgr4gh0000gn/T/tmpjmomkmfr/assets


Model is 984 bytes


In [100]:
# Save the file as a C source file
!xxd -i {model_path} > tinyml.cc